In [ ]:
import sys
import os

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy
import pandas as pd
import datetime
import copy

# Local files
__root__ = '../../'
module_path = os.path.abspath(os.path.join(__root__[:-1]))
if module_path not in sys.path:
    sys.path.append(f"{module_path}/py/")

import log_ret

import misc_functions as misc_fns

In [ ]:
B20_setup = dict()

B20_setup['results_dir'] = f"{__root__}results/"

B20_setup['datetime'] = dict()
B20_setup['datetime']['dt'] = {'w': datetime.datetime.now(datetime.timezone.utc)}
B20_setup['datetime']['str'] = {i: misc_fns.dt_str(B20_setup['datetime']['dt'][i]) for i in B20_setup['datetime']['dt']}


B20_setup['ticks'] = ['SPX', 'DJI', 'DAX', 'CAC', 'NKX', 'HSI']

B20_setup['a_q'] = np.array([float(f'{x:.5f}') for x in 0.0025*(1+np.array(range(60)))])

B20_setup['a_u'] = np.array([float(f'{x:.5f}') for x in 0.0125*(1+np.array(range(20)))])


B20_setup['mode_bound'] =   {
                                'bi': 'both',
                                'bi_d': ['both', 'left', 'right'],
                                'ci': 'both',
                                'ci_s': 'both',
                            }

B20_setup['dates'] = {
                        'all_start': pd.Timestamp("1959-10-02"),
                        #'all_end': pd.Timestamp("2020-11-20"),
                        'all_end': pd.Timestamp("2023-09-16"),
                        'train_end': pd.Timestamp("2008-09-01"),
                    }

In [ ]:
for x in B20_setup['a_q']:
    print(x)

In [ ]:
for x in B20_setup['a_u']:
    print(x)

In [ ]:
B20 = log_ret.log_ret(direct=__root__, ticks=B20_setup['ticks'])

In [ ]:

B20.set_tk(
                **B20_setup['dates'],
                cv_vol=["GARCH"], cv_p=1, cv_o=[0,1], cv_q=1, cv_dist=["norm", "t"],
                tt_a_u=B20_setup['a_u'], tt_dt_dt_fine=24,
                hb_dist=["norm", "t"], hb_bound=None, hb_mode_bound=B20_setup['mode_bound'],
                a_q=B20_setup['a_q'],
                #cv_set_am_stats=True, cv_set_eval=False, append_fit_cv_w=True, append_vrs_cv_w=False, cv_write_tick=False,
                #tt_i_0=0, tt_read_params=True, tt_fit_if_need=False,
                set_cv=False, set_tt=True,
                CT_Q=["binom", "UC", "CC", 'DQ_0', 'DQ_1', 'DQ_2', 'DQ_3', 'DQ_4'],
                CT_E=['ZMD'],
            )

In [ ]:
for tick in B20.tk:
    print(B20.tk[tick].pd.lr['p']['all'].index[0],
            B20.tk[tick].pd.lr['p']['train'].index[-1], 
            B20.tk[tick].pd.lr['p']['forecast'][0][0].index[0], 
            B20.tk[tick].pd.lr['p']['all'].index[-1])

In [ ]:
for fix_a_lambda in [True, False]:
    for i in range(0, B20_setup['a_u'].size):
        B20.set_tt_hb_mp_tick(write_tick=True, 
                            a_u_rng=[i], tt_read_params=True, tt_hb_read_params=False, tt_fit_if_need=False, tt_fix_a_lambda=fix_a_lambda, append_fit_tt_w=True, set_eval=True, append_fit_tt_hb_w=True, append_vrs_tt_hb_w=True)